Jetsim : Collect Images Locally
===

### Step **3** from README.md



---

### INITIAL SETUP

Turn ON **Donkeycar Simulator** before running the code below

Make sure you have done `conda activate donkey`

---


Create the racecar dictionary
===

---

Customize your car settings.

---

In [ ]:
# We are initilizaing the configuration file (Similar to myconfig.py)

GYM_DICT={
  'car':{
    #racer info
    'racer_name': 'Jetsim_NAME',    #CHANGE
    'bio' : 'info',                 #CHANGE
    'country' : 'US',
    "guid": "GO_JETSIM",
    #customize car
    'body_style' : 'f1',            #car01, donkey
    'body_rgb': (24, 43, 200),      #(R, G, B)
    'car_name' : 'Training_Jetsim', #Displays above car
    'font_size' : 50,
    },

    #connection type
    'default_connection': 'local',
    # 'default_connection': 'remote',

    'local_connection':{
      'scene_name': 'warren', # warren | thunderhill | mini_monaco | roboracingleague_1 | generated_track | generated_road | warehouse | sparkfun_avc | waveshare
######YOUR HOST COMPUTER IP ADDRESS######
      'host': '127.0.0.1', #UPDATE WITH COMPUTER IP ADDRESS HOSTING SIM
      'port': 9091,
      'artificial_latency': 0},

        
    'lidar':{
        'enabled': False,
        'deg_inc': 1, # Degree increment between each ray of the lidar
        'max_range': 30.0}, # Max range of the lidar laser
    }


Send dictionary to Triton Racer Gym interface
===

### Make sure to turn ON **Donkeycar Simulator** before running the code below

In [ ]:
import time
%cd resources
from gyminterface import GymInterface
%cd ..
gym = GymInterface(gym_config=GYM_DICT) #Load in the car based on default configurations
time.sleep(1)
imageSeen, x, y, z, speed, cte, lidar = gym.step(0.0, 0.0, 0.0, False) #gym.step(steering, throttle, brakes, reset)
time.sleep(1)
#imageSeen.shape only for checking returned image dim. should be 224x224

In [ ]:
#If the custom car setting did not load correctly, uncomment this to take car out of sim and then re-run the cell above.
gym.onShutdown()

---

This will initialize the **racecar in the simulator**. Check if the car **populated** in the sim.

---

There are 2 methods to control car and collect data. For COSMOS C11, Use *clickable*.
===

- **Method 1 / *clickable*** Click image to drive and collect data at the same time. 
- **Method 2 / keyboard*** Keyboard control and post processing images.

### Tip

- collect ~200 images minimum following on yellow lines.
- collect ~200 images minimum avoiding outer lines and cones.

Method 1 Image clickable and drive.
===

### 2 widgets will pop up:

- Clickable 'virtual' camera that takes a picture and drives the car to that direction.
- Click and hold 'virtual' joystick to manually control the car.

*Both windows can control the car.*

### Images will save in *clickable_images* folder.

### How to collect data
    Click the 'virtual' camera image in the direction you want the car to drive. Hold the click to pick up more speed. Click farther out to drive faster or closer to car to drive slower.
    If you need to correct your position without taking a picture use the **'virtual' joystick** to manually move the car in position.



In [ ]:
from ipycanvas import Canvas
from ipywidgets import Image
import cv2
import numpy as np
import os
import ipywidgets
from datetime import datetime


def handle_mouse_down(x, y): # Take a screenshot and drive the car in that direction
    global newImage
    x = int(x)
    y = int(y)
    gym.step((1.0 * (x / 224.0 - 0.5)), (-1.0 * (y / 224.0 - 0.8)), 0.0, False)

    now = str(datetime.now())
    now = now.replace(" ", "")
    now = now.replace(".", "")
    now = now.replace(":", "-")
    newName = str(x) + "_" + str(y) + "_" + str(now) + ".jpg"
    cv2.imwrite("./clickable_images/"+newName, newImage)

def handle_mouse_up(x, y):
    rewrite_canvas()

def rewrite_canvas(): #Load new image
    global newImage
    canvas.clear()
    imageSeen, cx, cy, cz, speed, cte, lidar = gym.step(0.0, 0.0, 0.0, False)
    newImage = np.copy(imageSeen)
    newImage[:, :, [2, 0]] = newImage[:, :, [0, 2]]
    cv2.imwrite("./clickable_images/DELETE.jpg", newImage)
    current = Image.from_file('./clickable_images/DELETE.jpg')
    canvas.draw_image(current, 0, 0)
    os.remove('./clickable_images/DELETE.jpg')

def manual_mouse_down(x, y):
    joystick.on_mouse_move(manual_move, remove=False)

def manual_mouse_up(x, y):
    joystick.on_mouse_move(manual_move, remove=True)
    gym.step(0.0, 0.0, 1.0, False)

def manual_out(x, y):
    gym.step(0.0, 0.0, 1.0, False)

def manual_move(x, y):
    gym.step((1.0 * (x / 224.0 - 0.5)), (-1.0 * (y / 224.0 - 0.5)), 0.0, False)



global newImage
time.sleep(1)
gym.step(0.0, 0.0, 0.0, True)
time.sleep(1)

dir = "clickable_images" #Make the directory for images to be stored in
try:
    os.makedirs(dir, exist_ok = False)
    print("Directory '%s' created successfully" % dir)
except OSError as error:
    print("Directory '%s' already exists" % dir)

print ("\n\n************************ CONTROLS ************************\n\nRepeatedly click path on image to move car.\nClick farther out to drive faster or closer to car to drive slower.\nHINT: find the optimal driving path.\n\n***********************************************************\n\n")

canvas = Canvas(width=224, height=224)
rewrite_canvas()
canvas.on_mouse_down(handle_mouse_down)
canvas.on_mouse_up(handle_mouse_up)

joystick = Canvas(width=224, height=224)
directions = Image.from_file('./resources/joystick.png')
joystick.draw_image(directions, 0, 0)
joystick.on_mouse_up(manual_mouse_up)
joystick.on_mouse_down(manual_mouse_down)
joystick.on_mouse_move(manual_move, remove=True)
joystick.on_mouse_out(manual_out)
joystick.stroke_style = 'blue'
joystick.stroke_rect(2, 2, 222, 222)


all_controls = ipywidgets.HBox([canvas, joystick])
display(all_controls)



In [ ]:
#reset car if needed
gym.step(0.0, 0.0, 0.0, True);

### Compress your clickable_images folder

In [ ]:
!zip -r clickable_images.zip clickable_images

---
### If finished - Verify *clickable_images* folder is populated with data. Exit this notebook.
### Proceed to **step 4.1** in README.md and transfer folders to Google Drive. Good luck!

---

In [ ]:
gym.onShutdown()

## EXIT NOTEBOOK AND GO TO STEP 4.1 IN THE README

Method 2 Keyboard control
===

### 2 windows will pop up:

- Live stream image from the 'virtual' camera.
- Pygame black square for keyboard inputs.

*Both windows are needed to control the car.*

### Images will save in *keyboard_images* folder.

    ************ CONTROLS ************

    Click the **black square** to activate controls:

    Arrow keys: DRIVE

    C : CAMERA (take 3 pictures)

    X : BRAKE

    Z : SHOW current car coordinates

    Red STOP button : 'kernel interrupt' to stop training loop.





In [ ]:
import cv2

def bgr8_to_jpeg(value, quality=100):
    return bytes(cv2.imencode('.jpg', value)[1])

import pygame, time
from pygame.locals import *
import numpy as np
import time
import os

counter = 1
steer = 0.0
throttle = 0.0
brakes = 0.0

pygame.init()
screen = pygame.display.set_mode((100, 100)) #black square
pygame.display.set_caption('Pygame Keyboard Test')
pygame.mouse.set_visible(0)

print ("\n\n************ CONTROLS ************\n\n   Arrow keys: DRIVE\n   Z : SHOW car coordinates\n   X : BRAKE\n   C : CAMERA (take 5 pictures)\n   Red STOP button : 'kernel interrupt' to stop training loop\n\n***********************************\n\n")

dir = "keyboard_images"
try:
    os.makedirs(dir, exist_ok = False)
    print("Directory '%s' created successfully" % dir)
except OSError as error:
    print("Directory '%s' already exists" % dir)
  
imageSeen, x, y, z, speed, cte, lidar = gym.step(0.0, 0.0, 0.0, False)
newArray = np.copy(imageSeen)
print(newArray.shape)
newArray[:, :, [2, 0]] = newArray[:, :, [0, 2]]

try:
    while True:
        keyState = pygame.key.get_pressed()
        if(len(keyState) != 0):
            if keyState[pygame.K_UP]:
                throttle = 0.5
            elif keyState[pygame.K_DOWN]:
                throttle = -1.0
            else:
                throttle = 0.0
            if keyState[pygame.K_RIGHT]:
                steer = 1.0
            elif keyState[pygame.K_LEFT]:
                steer = -1.0
            else:
                steer = 0.0
                
            if keyState[pygame.K_x]:
                brakes = 1000.0
            else:
                brakes = 0.0
                
            imageSeen, x, y, z, speed, cte, lidar = gym.step(steer, throttle, brakes, False)
            newArray = np.copy(imageSeen)
            newArray[:, :, [2, 0]] = newArray[:, :, [0, 2]]

            if keyState[pygame.K_c]:

                for i in range(3): #takes 3 pictures in one shot
                    
                    cv2.imwrite('./keyboard_images/' + 'data_point_' + "{:03d}".format(counter) + ".jpg", (newArray))
                    counter = counter + 1
                print(str(counter-1) + " image(s) captured")
                time.sleep(.3) #snap the photo and wait .3 seconds to allow finger to release button. While loop is too fast when you press B.
            if counter >= 999:
                print("TOO MANY PICTURES")
                gym.step(0.0, 0.0, 100.0, False)
                time.sleep(1)
                print('\nTRAINING STOPPED')
                gym.step(0.0, 0.0, 0.0, True);
                break

            if keyState[pygame.K_z]:
                print("x = "+str(x)+", z = "+str(z))
                time.sleep(.3)


            if keyState[pygame.K_v]:
                print(lidar)
                time.sleep(.3)
            
        else:
            throttle = 0.0
            steer = 0.0
            brakes = 0.0
            print('BRAKE')
            imageSeen, x, y, z, speed, cte, lidar = gym.step(0.0, 0.0, 100.0, False)
        pygame.event.pump()

        cv2.imshow("image", newArray)

except KeyboardInterrupt:
    
    #Resets steering and throttle to 0, applies brakes and resets for 1 second
    gym.step(0.0, 0.0, 100.0, False)
    time.sleep(1)
    print('\nTRAINING STOPPED')
    gym.step(0.0, 0.0, 0.0, True)

In [ ]:
gym.onShutdown()

---
### If finished - Click interrupt Jupyter kernel "STOP" button at top controls.
### Verify *keyboard_images* folder is populated with data. Exit this notebook.
### Proceed to *step 3.2* in README.md and run *postprocess_images.py* Good luck!

---